#Работа с API

In [1]:
import requests
import pandas as pd
from time import sleep

In [24]:
def fetch_paginated_data(endpoint, params=None):
    base_url = f"https://www.anapioficeandfire.com/api/{endpoint}"
    all_data = []
    page = 1
    page_size = 50

    while True:
        request_params = {
            'page': page,
            'pageSize': page_size,
            **(params or {})
        }

        response = requests.get(base_url, params=request_params)

        if response.status_code != 200:
            print(f"Ошибка {response.status_code} на странице {page}")
            break

        data = response.json()
        if not data:
            break

        all_data.extend(data)
        page += 1
        sleep(0.5)

    return all_data

books = fetch_paginated_data('books')
df_books = pd.DataFrame(books)

houses = fetch_paginated_data('houses')
df_houses = pd.DataFrame(houses)

houses_with_devis = fetch_paginated_data('houses', params={'hasWords': 'true'})
df_houses_with_devis = pd.DataFrame(houses_with_devis)

df_books.to_csv('Книги.csv', index=False)
df_houses.to_csv('Дома.csv', index=False)
df_houses_with_devis.to_csv('Дома с девизом.csv', index=False)

In [25]:
print("\nПримеры домов с девизом:")
print(df_houses_with_devis[['name', 'words']].head(10))


Примеры домов с девизом:
                             name                  words
0     House Allyrion of Godsgrace        No Foe May Pass
1                   House Ambrose          Never Resting
2        House Arryn of the Eyrie       As High as Honor
3        House Ashford of Ashford  Our Sun Shines Bright
4  House Baratheon of Storm's End       Ours is the Fury
5     House Beesbury of Honeyholt       Beware our Sting
6   House Bolton of the Dreadfort   Our Blades are Sharp
7   House Buckwell of the Antlers      Pride and Purpose
8      House Bulwer of Blackcrown  Death Before Disgrace
9        House Caron of Nightsong       No Song so Sweet


In [15]:
print("\nПримеры книг:")
print(df_books[['name', 'authors']].head(10))


Примеры книг:
                         name                authors
0           A Game of Thrones  [George R. R. Martin]
1            A Clash of Kings  [George R. R. Martin]
2           A Storm of Swords  [George R. R. Martin]
3            The Hedge Knight  [George R. R. Martin]
4           A Feast for Crows  [George R. R. Martin]
5             The Sworn Sword  [George R. R. Martin]
6          The Mystery Knight  [George R. R. Martin]
7        A Dance with Dragons  [George R. R. Martin]
8  The Princess and the Queen  [George R. R. Martin]
9            The Rogue Prince  [George R. R. Martin]


#Работа с БД

In [20]:
import psycopg2
import pandas as pd

In [26]:
def connect_to_db():
    """Подключение к публичной БД EBI"""
    try:
        conn = psycopg2.connect(
            host="hh-pgsql-public.ebi.ac.uk",
            port=5432,
            database="pfmegrnargs",
            user="reader",
            password="NWDMCE5xdipIjRrp"
        )
        print("ОК")
        return conn
    except Exception as e:
        print(f"Ошибка подключения: {e}")
        return None

def db_operations():
    conn = connect_to_db()
    if not conn:
        return

    try:
        with conn.cursor() as cursor:
            cursor.execute("SELECT * FROM rnc_database LIMIT 10")
            rows = cursor.fetchall()
            colnames = [desc[0] for desc in cursor.description]

            df_full = pd.DataFrame(rows, columns=colnames)
            print("\nПервые 10 строк (все столбцы):")
            print(df_full.head())

            df_full.to_csv('10 строк.csv', index=False)

        with conn.cursor() as cursor:
            cursor.execute(
                "SELECT display_name, num_sequences, num_organisms, url "
                "FROM rnc_database LIMIT 10"
            )
            rows = cursor.fetchall()

            df_specific = pd.DataFrame(
                rows,
                columns=['display_name', 'num_sequences', 'num_organisms', 'url']
            )

            print("\nСпецифичные столбцы для 10 строк:")
            print(df_specific)

            df_specific.to_csv('Новые 10 строк.csv', index=False)

        return df_full, df_specific

    except Exception as e:
        print(f"Ошибка при выполнении запроса: {e}")
    finally:
        conn.close()
        print("Соединение с базой данных закрыто")

if __name__ == "__main__":
    df_full, df_specific = db_operations()

ОК

Первые 10 строк (все столбцы):
   id  timestamp userstamp    descr  current_release full_descr alive  \
0  21 2017-05-02    RNACEN  NONCODE              146    NONCODE     Y   
1   5 2017-05-17    RNACEN     VEGA               98       VEGA     N   
2  26 2017-05-01    RNACEN  GENCODE              450    GENCODE     N   
3   1 2017-05-01    RNACEN      ENA              968        ENA     Y   
4  14 2017-05-01    RNACEN     TAIR              982       TAIR     Y   

  for_release display_name project_id  avg_length  min_length  max_length  \
0                  NONCODE                 1130.0       201.0    244296.0   
1                     VEGA  PRJEB4568         NaN         NaN         NaN   
2                  GENCODE                  889.0        32.0    205012.0   
3                      ENA                  412.0        10.0    900074.0   
4                     TAIR   PRJ_TAIR       384.0        19.0      6227.0   

   num_sequences  num_organisms  \
0         234669            